In [48]:
import nltk
import re
import pandas as pd 
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.collocations import *
from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer 
from autocorrect import Speller
from gensim.models import Word2Vec
from gensim.models.phrases import Phraser, Phrases
#from geotext import GeoText

#import spacy
#python -m spacy download en_core_web_sm
pattern = re.compile('^[^a-z]+$')

resumeDS = pd.read_csv('resume_dataset.csv', encoding='utf-8')
#print out job title and numbers of resume associated to the job title
print (resumeDS['Category'].value_counts())


Java Developer               14
Database                     11
HR                           11
Data Science                 10
Advocate                     10
DevOps Engineer               7
Testing                       7
Automation Testing            7
Hadoop                        7
DotNet Developer              7
Civil Engineer                6
Business Analyst              6
Python Developer              6
SAP Developer                 6
Health and fitness            6
Arts                          6
ETL Developer                 5
Network Security Engineer     5
Mechanical Engineer           5
Electrical Engineering        5
Sales                         5
Blockchain                    5
Web Designing                 5
Operations Manager            4
PMO                           3
Name: Category, dtype: int64


In [49]:
def alpha_filter(w):
  # pattern to match a word of non-alphabetical characters
    if (pattern.match(w)):
        return True
    else:
        return False


In [50]:
sentences = resumeDS['Resume'].values
stopwords = nltk.corpus.stopwords.words('english')
stopwords.append("months")
monthstopwords = ["january", "february", "march", "april", "may", "june", "july", 
"august", "september", "octorber", "november", "december"]
stopwords.extend(monthstopwords)


In [51]:
sentences[0]

'Skills * Programming Languages: Python (pandas, numpy, scipy, scikit-learn, matplotlib), Sql, Java, JavaScript/JQuery. * Machine learning: Regression, SVM, NaÃ¯ve Bayes, KNN, Random Forest, Decision Trees, Boosting techniques, Cluster Analysis, Word Embedding, Sentiment Analysis, Natural Language processing, Dimensionality reduction, Topic Modelling (LDA, NMF), PCA & Neural Nets. * Database Visualizations: Mysql, SqlServer, Cassandra, Hbase, ElasticSearch D3.js, DC.js, Plotly, kibana, matplotlib, ggplot, Tableau. * Others: Regular Expression, HTML, CSS, Angular 6, Logstash, Kafka, Python Flask, Git, Docker, computer vision - Open CV and understanding of Deep learning.Education Details \r\nData Science Assurance AssociateData Science Assurance Associate - Ernst & Young LLPSkill Details \r\nJAVASCRIPT- Exprience - 24 months\r\njQuery- Exprience - 24 months\r\nPython- Exprience - 24 monthsCompany Details \r\ncompany - Ernst & Young LLP\r\ndescription - Fraud Investigations and Dispute Se

In [52]:
#Removing proper nouns like Nikhil, India etc using POS tagging
from nltk.tag import pos_tag
totalNounList = []

def listOfNouns(sentence):
    tagged_sent = pos_tag(sentence.split())
    propernouns = [word for word,pos in tagged_sent if pos == 'NNP']
    totalNounList.extend(propernouns)

In [53]:
totalResumeWords = []

for i in range (0, len(sentences)):
    removeSpace = re.sub(' +', ' ',sentences[i])
    
    text = removeSpace
    listOfNouns(text)
    tokenizeText = nltk.word_tokenize(text)
    for w in tokenizeText:
        if w.lower() not in stopwords and not alpha_filter(w):
            totalResumeWords.append(w)
            
totalResumeWords[:20]

['Skills',
 'Programming',
 'Languages',
 'Python',
 'pandas',
 'numpy',
 'scipy',
 'scikit-learn',
 'matplotlib',
 'Sql',
 'Java',
 'JavaScript/JQuery',
 'Machine',
 'learning',
 'Regression',
 'NaÃ¯ve',
 'Bayes',
 'Random',
 'Forest',
 'Decision']

In [54]:
#Checking first 20 Proper nouns
totalNounList[:20]

['Skills',
 '*',
 'Programming',
 'Languages:',
 'Python',
 '(pandas,',
 'Sql,',
 'Java,',
 'JavaScript/JQuery.',
 '*',
 'Machine',
 'Regression,',
 'SVM,',
 'NaÃ¯ve',
 'Bayes,',
 'KNN,',
 'Random',
 'Forest,',
 'Decision',
 'Trees,']

In [55]:
lowerResumeWords = [w.lower() for w in totalResumeWords if len(w)>=3]
lowerResumeWords

['skills',
 'programming',
 'languages',
 'python',
 'pandas',
 'numpy',
 'scipy',
 'scikit-learn',
 'matplotlib',
 'sql',
 'java',
 'javascript/jquery',
 'machine',
 'learning',
 'regression',
 'naã¯ve',
 'bayes',
 'random',
 'forest',
 'decision',
 'trees',
 'boosting',
 'techniques',
 'cluster',
 'analysis',
 'word',
 'embedding',
 'sentiment',
 'analysis',
 'natural',
 'language',
 'processing',
 'dimensionality',
 'reduction',
 'topic',
 'modelling',
 'neural',
 'nets',
 'database',
 'visualizations',
 'mysql',
 'sqlserver',
 'cassandra',
 'hbase',
 'elasticsearch',
 'd3.js',
 'dc.js',
 'plotly',
 'kibana',
 'matplotlib',
 'ggplot',
 'tableau',
 'others',
 'regular',
 'expression',
 'angular',
 'logstash',
 'kafka',
 'python',
 'flask',
 'git',
 'docker',
 'computer',
 'vision',
 'open',
 'understanding',
 'deep',
 'learning.education',
 'details',
 'data',
 'science',
 'assurance',
 'associatedata',
 'science',
 'assurance',
 'associate',
 'ernst',
 'young',
 'llpskill',
 'detail

In [56]:
lemmatizer = WordNetLemmatizer()

In [57]:
#lemmatize words
lemmatizeWords = [lemmatizer.lemmatize(w) for w in lowerResumeWords]
len(lemmatizeWords)

43538

In [58]:
wordFreq = FreqDist(lemmatizeWords)
mostCommon = wordFreq.most_common(20)

spell = Speller(lang='en')
mostCommon_correct = [(spell(w),t) for w,t in mostCommon]

for item in mostCommon_correct:
	print (item)


('experience', 634)
('company', 615)
('project', 589)
('description', 551)
('detail', 537)
('data', 365)
('database', 313)
('system', 310)
('management', 300)
('team', 296)
('year', 260)
('server', 238)
('maharashtra', 223)
('test', 217)
('client', 215)
('using', 210)
('report', 209)
('le', 206)
('technology', 205)
('application', 205)


In [59]:
top20words = [w[0] for w in mostCommon_correct]
resumebigrams = list(nltk.bigrams(lowerResumeWords))


In [60]:
resumebigrams[:5]

[('skills', 'programming'),
 ('programming', 'languages'),
 ('languages', 'python'),
 ('python', 'pandas'),
 ('pandas', 'numpy')]

In [61]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
#finder = BigramCollocationFinder.from_words(top20words)
finder = BigramCollocationFinder.from_words(lowerResumeWords)

scored = finder.score_ngrams(bigram_measures.raw_freq)
for bscore in scored[:30]:
    print (bscore)

(('exprience', 'exprience'), 0.0057880472231154395)
(('details', 'company'), 0.0048693095686526714)
(('exprience', 'less'), 0.004708530479121687)
(('less', 'year'), 0.004708530479121687)
(('monthscompany', 'details'), 0.002939960494280858)
(('education', 'details'), 0.002457623225687905)
(('exprience', 'monthscompany'), 0.0019752859570949517)
(('ltd', 'description'), 0.0019752859570949517)
(('pvt', 'ltd'), 0.001814506867563967)
(('details', 'exprience'), 0.0017456015434792595)
(('company', 'description'), 0.0013321695989710139)
(('pune', 'maharashtra'), 0.0013092011576094447)
(('mumbai', 'maharashtra'), 0.0012173273921631679)
(('description', 'company'), 0.0009646745371859065)
(('year', 'monthscompany'), 0.0009646745371859065)
(('company', 'details'), 0.0009417060958243373)
(('description', 'project'), 0.0009417060958243373)
(('management', 'system'), 0.0008728007717396297)
(('project', 'description'), 0.0008728007717396297)
(('project', 'name'), 0.0008728007717396297)
(('year', 'expri

In [62]:
# applying a filter with minimum frequency of 3 to the bigram collection finder
finder.apply_freq_filter(3)
scored = finder.score_ngrams(bigram_measures.raw_freq)
for bscore in scored[:20]:
    print (bscore)

(('exprience', 'exprience'), 0.0057880472231154395)
(('details', 'company'), 0.0048693095686526714)
(('exprience', 'less'), 0.004708530479121687)
(('less', 'year'), 0.004708530479121687)
(('monthscompany', 'details'), 0.002939960494280858)
(('education', 'details'), 0.002457623225687905)
(('exprience', 'monthscompany'), 0.0019752859570949517)
(('ltd', 'description'), 0.0019752859570949517)
(('pvt', 'ltd'), 0.001814506867563967)
(('details', 'exprience'), 0.0017456015434792595)
(('company', 'description'), 0.0013321695989710139)
(('pune', 'maharashtra'), 0.0013092011576094447)
(('mumbai', 'maharashtra'), 0.0012173273921631679)
(('description', 'company'), 0.0009646745371859065)
(('year', 'monthscompany'), 0.0009646745371859065)
(('company', 'details'), 0.0009417060958243373)
(('description', 'project'), 0.0009417060958243373)
(('management', 'system'), 0.0008728007717396297)
(('project', 'description'), 0.0008728007717396297)
(('project', 'name'), 0.0008728007717396297)


In [63]:
#trigrams
trigrams=ngrams(lowerResumeWords,3)
trigramslist = list(trigrams)
trigramslist[:5]

[('skills', 'programming', 'languages'),
 ('programming', 'languages', 'python'),
 ('languages', 'python', 'pandas'),
 ('python', 'pandas', 'numpy'),
 ('pandas', 'numpy', 'scipy')]

In [64]:
#unigrams
unigrams = list(ngrams(lowerResumeWords,1))
unigrams[:5]

[('skills',), ('programming',), ('languages',), ('python',), ('pandas',)]

In [95]:
#creating the model below using unigrams for further prediction
import gensim
modeluni=gensim.models.Word2Vec(unigrams,size=5000,min_count=2,workers=4,window=4)
modeluni.save("final.modeluni")
wrds=list(modeluni.wv.vocab)
print(len(wrds))

3607


In [96]:
#creating the model below using bigrams for further prediction
modelbi=gensim.models.Word2Vec(resumebigrams,size=5000,min_count=2,workers=4,window=4)
modelbi.save("final.modelbi")
wrds=list(modelbi.wv.vocab)
print(len(wrds))

7226


In [97]:
#creating the model below using trigrams for further prediction
modeltri=gensim.models.Word2Vec(trigramslist,size=5000,min_count=2,workers=4,window=4)
modeltri.save("final.modeltri")
wrds=list(modeltri.wv.vocab)
print(len(wrds))

7226


In [98]:
#creating the model below using uni + bi +trigrams for further prediction
allgram = []
allgram.extend(unigrams)
allgram.extend(resumebigrams)
allgram.extend(trigramslist)
modelall=gensim.models.Word2Vec(allgram,size=5000,min_count=2,workers=4,window=4)
modelall.save("final.modelall")
wrds=list(modelall.wv.vocab)
print(len(wrds))

7226


In [66]:
wrds

['skills',
 'programming',
 'languages',
 'python',
 'pandas',
 'numpy',
 'scipy',
 'scikit-learn',
 'matplotlib',
 'sql',
 'java',
 'javascript/jquery',
 'machine',
 'learning',
 'regression',
 'naã¯ve',
 'bayes',
 'random',
 'forest',
 'decision',
 'trees',
 'boosting',
 'techniques',
 'cluster',
 'analysis',
 'word',
 'embedding',
 'sentiment',
 'natural',
 'language',
 'processing',
 'dimensionality',
 'reduction',
 'topic',
 'modelling',
 'neural',
 'nets',
 'database',
 'visualizations',
 'mysql',
 'sqlserver',
 'cassandra',
 'hbase',
 'elasticsearch',
 'd3.js',
 'dc.js',
 'plotly',
 'kibana',
 'ggplot',
 'tableau',
 'others',
 'regular',
 'expression',
 'angular',
 'logstash',
 'kafka',
 'flask',
 'git',
 'docker',
 'computer',
 'vision',
 'open',
 'understanding',
 'deep',
 'learning.education',
 'details',
 'data',
 'science',
 'assurance',
 'associatedata',
 'associate',
 'ernst',
 'young',
 'llpskill',
 'exprience',
 'jquery-',
 'python-',
 'monthscompany',
 'company',
 'des

In [67]:
z=model.wv.most_similar("machine")
print(z)

[('data', 0.9976101517677307), ('using', 0.99748694896698), ('like', 0.9974251389503479), ('new', 0.9974243640899658), ('various', 0.9974197149276733), ('etc', 0.9974147081375122), ('client', 0.9974001049995422), ('project', 0.9973745346069336), ('work', 0.9973597526550293), ('database', 0.9973540306091309)]


In [68]:
#z=model.wv.most_similar("machine learning")
#z=model.wv.most_similar("machine_learning")
#z=model.wv.most_similar("machine")
#z=model.wv.most_similar("learning")

In [99]:
# pointwise mutual information
finder3 = BigramCollocationFinder.from_words(lowerResumeWords)
scored3 = finder3.score_ngrams(bigram_measures.pmi)
for bscore in scored3[:20]:
    print (bscore)

(("'acid", "'midi"), 15.40998751568481)
(('/development', 'collabnet'), 15.40998751568481)
(('/six', 'sigma/quality'), 15.40998751568481)
(('1-jan', '24th'), 15.40998751568481)
(('1.autocad', '2.pro'), 15.40998751568481)
(('132kv', 'switch-yard'), 15.40998751568481)
(('1800s', '1900s'), 15.40998751568481)
(('1900s', '2600s'), 15.40998751568481)
(('2.pro', '3.catia'), 15.40998751568481)
(('2.regulated', 'subcontractor'), 15.40998751568481)
(('24-hour', 'turnaround'), 15.40998751568481)
(('28xx', '19xx'), 15.40998751568481)
(('29xx', '28xx'), 15.40998751568481)
(('3.catia', '4.word'), 15.40998751568481)
(('3600s', '3800s'), 15.40998751568481)
(('3700s', '4850s'), 15.40998751568481)
(('3800s', '7200s'), 15.40998751568481)
(('aakruti', 'kalepadal'), 15.40998751568481)
(('abasaheb', 'garware'), 15.40998751568481)
(('abridged', 'decrease'), 15.40998751568481)
